<a href="https://colab.research.google.com/github/MauricioLucas/pylibrary/blob/master/DeepFake_1_Imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLearning na Prática (siga nossos tutoriais no [youtube](https://www.youtube.com/channel/UCGzmqer9amvRWgHRmNuj6LA)) 
## **Criação de Deepfake com 1 imagem**

** **Primeiro devemos baixar os arquivos fonte do modelo**

In [ ]:
!git clone https://github.com/deeplearningnapratica/deepfake1imagem

Cloning into 'deepfake1imagem'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 232 (delta 120), reused 232 (delta 120), pack-reused 0
Receiving objects: 100% (232/232), 71.45 MiB | 37.99 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
cd deepfake1imagem

/content/deepfake1imagem


** **Mondar seu drive pessoal do Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


**Adicione a pasta https://drive.google.com/open?id=1IBpce06RJiO-yBd_i4hHyT2CMypSQKEN  para o seu google drive. Dica secreta: Shift + Z**

**Carregue o vídeo original e a imagem de destino**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/DeepFake1Imagem/moro.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/DeepFake1Imagem/00.mp4')


#O modelo atual só funciona com as dimensões 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Criar modelo e carregar checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/DeepFake1Imagem/vox-cpk.pth.tar')

**Fazer a mágica acontecer!**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../arquivo_gerado.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 265/265 [00:12<00:00, 21.81it/s]


**Na célula acima, usamos o deslocamento relativo para animar os objetos. Em vez disso, podemos usar coordenadas absolutas, mas dessa maneira todas as proporções de objetos serão herdadas do vídeo guia. Por exemplo, o corte de cabelo de Putin será estendido para corresponder ao corte de cabelo de Trump. Veja abaixo:**

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 265/265 [00:12<00:00, 22.03it/s]


**Você precisa utilizar um vídeo com a dimensão de 256x256. Existem várias formas de ajustar para esta dimensão. Você pode inclusive usar alguma ferramenta de gravação de tela. Tem uma forma mais inteligente de fazer isso, usando a detecção de rosto. Vou ensinar isso no canal em breve ([assina lá](https://www.youtube.com/channel/UCGzmqer9amvRWgHRmNuj6LA)).** 


Este tutorial só foi possível graças ao trabalho de Aliaksandr Siarohin. [Veja o tutorial em inglês](https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb?authuser=3#scrollTo=IcMX7ueZO0Oa) 